In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
print(sys.path)
sys.path.append('/home/lankenau/isotools/src')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

path = '/path/to/data'
alignment_path = 'alignment_v45'
genome_file = 'GRCh38.p14.genome.fa'
genome_path = os.path.join(path, 'gencode_human/version_45', genome_file)

In [ ]:
metadata_file = 'reads/metadata_tissue.tsv'
metadata = pd.read_csv(os.path.join(path, metadata_file), sep='\t')
metadata

In [7]:
import logging
from isotools import Transcriptome
from isotools import __version__ as isotools_version
# set up logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)
logger=logging.getLogger('isotools')
logger.info(f'This is isotools version {isotools_version}')

INFO:This is isotools version 0.3.5rc10


In [ ]:
annotation_file=os.path.join(path, 'gencode_human/version_45', 'gencode.v45.chr_patch_hapl_scaff.annotation_sorted.gff3.gz')
#create the IsoTools transcriptome object from the reference annotation
isoseq=Transcriptome.from_reference(annotation_file)

In [ ]:
for i, row in metadata.iterrows():
    sample_name = row['sample ID']
    # file is the full (wrong) path, we just need the filename without the extension
    sample_file = os.path.join(path, alignment_path, row['file'].split('/')[-1].split('.')[0] + '_aligned.bam')
    if not os.path.exists(sample_file):
        logger.error(f'File {sample_file} does not exist')
        continue
    group = row['group']
    isoseq.add_sample_from_bam(fn=sample_file, sample_name=sample_name, group=group, strictness=20)
isoseq.sample_table

In [10]:
# compute qc metrics
isoseq.add_qc_metrics(genome_path)
# add ORF predictions
isoseq.add_orf_prediction(genome_path)

100%|██████████| 540730/540730 [50:30<00:00, 178.43genes/s]


In [11]:
isoseq.save('results/isoseq_v45_strict.pkl')

INFO:saving transcriptome to results/isoseq_v45_strict.pkl


In [12]:
isoseq.write_gtf(f'results/isoseq_v45_strict.gtf',
                 source='isoseq', min_coverage=5,
                 gzip=False,
                 query="")

INFO:writing gtf file to results/isoseq_v45_strict.gtf


In [13]:
# export transcript table with the same filter criteria:
transcript_tab=isoseq.transcript_table( groups=isoseq.groups(),tpm=True,coverage=True,
                                       min_coverage=5, progress_bar=True,
                                       query="")
# write to csv file
transcript_tab.to_csv(f'{path}/demonstration_dataset_substantial_transcripts.csv',
                      index=False, sep='\t')

transcript_tab.head()

100%|██████████| 540730/540730 [00:31<00:00, 17152.18genes/s]


,chr,transcript_start,transcript_end,strand,gene_id,gene_name,transcript_nr,transcript_length,num_exons,exon_starts,...,ovary_sum_coverage,vessel_sum_coverage,aorta_sum_tpm,brain_sum_tpm,colon_sum_tpm,heart_sum_tpm,lung_sum_tpm,muscle_sum_tpm,ovary_sum_tpm,vessel_sum_tpm
0,GL000194.1,61662,62921,-,PB_novel_188266,PB_novel_188266,0,1259,1,61662,...,2,4,0.320567,0.58893,0.391436,0.771620,0.444486,0.00000,0.584430,2.083491
1,GL000194.1,53591,55442,-,ENSG00000277400.1,ENSG00000277400,0,1851,1,53591,...,3,3,2.243966,0.19631,1.957179,1.043957,1.926105,0.40793,0.876645,1.562618
2,GL000194.1,53591,115065,-,ENSG00000277400.1,ENSG00000277400,1,2224,3,"53591,112791,114985",...,5,3,1.602833,0.58893,1.565743,0.726231,1.629782,0.40793,1.461076,1.562618
3,GL000194.1,53591,115065,-,ENSG00000277400.1,ENSG00000277400,3,1439,4,"53591,54677,112791,114985",...,3,0,0.320567,0.00000,0.391436,0.000000,0.148162,0.00000,0.876645,0.000000
4,GL000194.1,53591,115065,-,ENSG00000277400.1,ENSG00000277400,4,2224,3,"53591,112791,114985",...,8,0,1.602833,0.00000,0.391436,0.635452,0.740810,0.00000,2.337721,0.000000
